### Recommender system with retail data

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv("/home/predator/Desktop/sales_history.csv",usecols=['id', 'customer_id', 'occurence', 'cost', 'level_one_id',
       'level_two_id', 'level_three_id', 'item_id', 'shop',
       'gender', 'birthdate'])


/home/predator/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(5)

,id,customer_id,occurence,cost,level_one_id,level_two_id,level_three_id,item_id,shop,gender,birthdate
0,2031214635,79085,2017-01-01 12:07:12.000000,10995.0,5.637149e+09,5.637149e+09,5.637149e+09,856539,11,f,NaN
1,2031214636,4854,2017-01-01 12:33:40.000000,8070.0,5.637149e+09,5.637149e+09,5.637149e+09,804602,5,m,1984-02-10
2,2031214637,4854,2017-01-01 12:33:40.000000,7833.0,5.637149e+09,5.637149e+09,5.637149e+09,811382,5,m,1984-02-10
3,2031214638,4854,2017-01-01 12:33:40.000000,7833.0,5.637149e+09,5.637149e+09,5.637149e+09,892613,5,m,1984-02-10
4,2031214639,4854,2017-01-01 12:33:40.000000,872.0,5.637151e+09,5.637308e+09,5.637308e+09,782090,5,m,1984-02-10


In [4]:
df.drop(['id','shop','gender','birthdate'],axis=1,inplace=True)

In [5]:
df.head()

,customer_id,occurence,cost,level_one_id,level_two_id,level_three_id,item_id
0,79085,2017-01-01 12:07:12.000000,10995.0,5.637149e+09,5.637149e+09,5.637149e+09,856539
1,4854,2017-01-01 12:33:40.000000,8070.0,5.637149e+09,5.637149e+09,5.637149e+09,804602
2,4854,2017-01-01 12:33:40.000000,7833.0,5.637149e+09,5.637149e+09,5.637149e+09,811382
3,4854,2017-01-01 12:33:40.000000,7833.0,5.637149e+09,5.637149e+09,5.637149e+09,892613
4,4854,2017-01-01 12:33:40.000000,872.0,5.637151e+09,5.637308e+09,5.637308e+09,782090


In [6]:
df['occurence'] = pd.to_datetime(df['occurence'])

In [7]:
df.dropna(inplace=True)

###  Для начала сделаем классификацию людей и сделаем чистку данных.

In [8]:
item_bought_time = df.groupby(['level_three_id'])['occurence'].count()

In [11]:
ids = item_bought_time>100

In [12]:
ids

level_three_id
5.637149e+09    False
5.637149e+09     True
5.637149e+09     True
5.637149e+09    False
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09    False
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09    False
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
5.637149e+09     True
                ...  
5.637297e+09     True
5.637297e+09     True
5.637297e+09     True
5.637297e+09     True
5.637298e+09     True
5.637298e+09     True
5.637301e+09    False
5.637304e+09     True
5.637305e+09    False
5.637307e+09     True
5.637307e+09     True
5.637308e+09     True
5.637308e+09     True
5.637308e+09    F

In [13]:
df = df[~df['level_three_id'].isin(ids[-ids].index)]

In [14]:
customer_transaction_count = df.groupby(['customer_id'])['occurence'].count()>100

In [15]:
df.shape

(12453189, 7)

In [16]:
df = df[~df['customer_id'].isin(customer_transaction_count[-customer_transaction_count].index)]

In [19]:
df.shape

(5133962, 7)

In [20]:
PRESENT = dt.datetime(2019,6,16)

In [21]:
rfm= df.groupby(['customer_id']).agg({'occurence': lambda date: (PRESENT - date.max()).days,
                                      'cost': lambda price: price.sum(),
                                      'item_id': lambda occurence: occurence.count()})
rfm.columns=['recency','monetary','frequency']
rfm=rfm.reset_index(level=['customer_id'])

In [22]:
quartiles = rfm.quantile(q=[0.20,0.40,0.60,0.80])

In [23]:
quartiles

,customer_id,recency,monetary,frequency
0.2,31201.0,13.0,261101.0,117.0
0.4,82625.0,20.0,389690.0,139.0
0.6,274333.0,33.0,556709.0,174.0
0.8,413054.0,62.0,863325.0,247.0


In [24]:
quartiles=quartiles.to_dict()
quartiles

{'customer_id': {0.2: 31201.0, 0.4: 82625.0, 0.6: 274333.0, 0.8: 413054.0},
 'recency': {0.2: 13.0, 0.4: 20.0, 0.6: 33.0, 0.8: 62.0},
 'monetary': {0.2: 261101.0, 0.4: 389690.0, 0.6: 556709.0, 0.8: 863325.0},
 'frequency': {0.2: 117.0, 0.4: 139.0, 0.6: 174.0, 0.8: 247.0}}

In [25]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1
## for Frequency and Monetary value 

def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5

In [26]:
rfm['R_Quartile'] = rfm['recency'].apply(RClass, args=('recency',quartiles,))
rfm['F_Quartile'] = rfm['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfm['M_Quartile'] = rfm['monetary'].apply(FMClass, args=('monetary',quartiles,))

In [27]:
rfm['Ranking'] = (rfm['R_Quartile'] + rfm['F_Quartile'] +rfm['M_Quartile'])/3

In [28]:
rfm.head()

,customer_id,recency,monetary,frequency,R_Quartile,F_Quartile,M_Quartile,Ranking
0,1,55,298474.000,150,2,3,2,2.333333
1,37,9,1078347.000,476,5,5,5,5.000000
2,54,9,905889.404,315,5,5,5,5.000000
3,156,36,242943.000,102,2,1,1,1.333333
4,165,12,756449.000,227,5,4,4,4.333333


###  Выбираем отсюда только лояльных клиентов, то есть тех ребят, у которых высокий балл

In [29]:
rfm_tiny = rfm[rfm['Ranking']>=3.5]

In [30]:
print(rfm_tiny['customer_id'].nunique())
print(rfm['customer_id'].nunique())

8455
24986


In [31]:
df1 = pd.merge(pd.DataFrame(rfm_tiny['customer_id']),df)

In [32]:
df.shape

(5133962, 7)

In [33]:
df1.shape

(2762248, 7)

In [34]:
rfm= df1.groupby(['level_three_id','customer_id']).agg({'occurence': lambda date: (PRESENT - date.max()).days,
                                        'cost': lambda price: price.sum()})


In [35]:
rfm['frequency']=df.groupby(['level_three_id','customer_id'])['cost'].count()

In [36]:
rfm.columns=['recency','monetary','frequency']

In [37]:
rfm=rfm.reset_index(level=['customer_id'])

In [38]:
rfm.head()

,customer_id,recency,monetary,frequency
level_three_id,,,,
5.637149e+09,232,652,630.0,2
5.637149e+09,312,693,207.0,1
5.637149e+09,426,841,630.0,2
5.637149e+09,588,827,261.0,1
5.637149e+09,685,818,332.0,1


In [39]:
quartiles = rfm.quantile(q=[0.20,0.40,0.60,0.80])

In [40]:
quartiles=quartiles.to_dict()
quartiles

{'customer_id': {0.2: 20304.0, 0.4: 63876.0, 0.6: 134862.0, 0.8: 404039.0},
 'recency': {0.2: 104.0, 0.4: 225.0, 0.6: 384.0, 0.8: 588.0},
 'monetary': {0.2: 992.0, 0.4: 2280.0, 0.6: 4750.0, 0.8: 11855.0},
 'frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 2.0, 0.8: 4.0}}

In [41]:
## for Recency 

def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1
## for Frequency and Monetary value 

def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5

In [42]:

rfm['R_Quartile'] = rfm['recency'].apply(RClass, args=('recency',quartiles,))
rfm['F_Quartile'] = rfm['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfm['M_Quartile'] = rfm['monetary'].apply(FMClass, args=('monetary',quartiles,))

In [43]:
rfm['Ranking'] = (rfm['R_Quartile'] + rfm['F_Quartile'] +rfm['M_Quartile'])/3

In [44]:
rfm.head()

,customer_id,recency,monetary,frequency,R_Quartile,F_Quartile,M_Quartile,Ranking
level_three_id,,,,,,,,
5.637149e+09,232,652,630.0,2,1,3,1,1.666667
5.637149e+09,312,693,207.0,1,1,1,1,1.000000
5.637149e+09,426,841,630.0,2,1,3,1,1.666667
5.637149e+09,588,827,261.0,1,1,1,1,1.000000
5.637149e+09,685,818,332.0,1,1,1,1,1.000000


In [45]:
result=rfm.drop(['R_Quartile','F_Quartile','M_Quartile','recency','monetary','frequency'],axis=1)
result

,customer_id,Ranking
level_three_id,,
5.637149e+09,232,1.666667
5.637149e+09,312,1.000000
5.637149e+09,426,1.666667
5.637149e+09,588,1.000000
5.637149e+09,685,1.000000
5.637149e+09,1030,1.000000
5.637149e+09,1369,1.333333
5.637149e+09,1438,1.333333
5.637149e+09,2157,1.000000


In [46]:
total = result.reset_index('level_three_id')

In [47]:
total['Ranking'] = round(total['Ranking'])

In [48]:
total

,level_three_id,customer_id,Ranking
0,5.637149e+09,232,2.0
1,5.637149e+09,312,1.0
2,5.637149e+09,426,2.0
3,5.637149e+09,588,1.0
4,5.637149e+09,685,1.0
5,5.637149e+09,1030,1.0
6,5.637149e+09,1369,1.0
7,5.637149e+09,1438,1.0
8,5.637149e+09,2157,1.0
9,5.637149e+09,2311,1.0


In [49]:
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader

In [50]:
total.head()

,level_three_id,customer_id,Ranking
0,5.637149e+09,232,2.0
1,5.637149e+09,312,1.0
2,5.637149e+09,426,2.0
3,5.637149e+09,588,1.0
4,5.637149e+09,685,1.0


In [51]:
reader = Reader(rating_scale=(1,5))

In [52]:
data = Dataset.load_from_df(total[['customer_id','level_three_id','Ranking']],reader)

In [53]:
algo = SVD()

In [ ]:
cross_validate(algo,data,measures=['RMSE','MAE'],cv=3,verbose=True,n_jobs=-1)

In [48]:
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

In [49]:
trainset,testset = train_test_split(data,test_size=.25)

In [50]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9738


0.9738079422186677

In [51]:
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions,verbose=True)

RMSE: 0.9808
RMSE: 0.9788
RMSE: 0.9812


In [52]:
from surprise.model_selection import GridSearchCV

In [53]:
param_grid = {
    'n_epochs':[5,10],
    'lr_all':[0.002,0.005],
    'reg_all':[0.4,0.6]
}
gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)

In [56]:
gs.best_score['rmse']

0.9901913020197165

In [57]:
gs.best_params

{'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4},
 'mae': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}